https://projecteuler.net/problem=31

In [12]:
# idea: running tally of descending sequences of coins with sum <= 200.
# Exclude coin value of 1p, imagining we are padding out with 1s

import collections

def ways_to_get_sum(max_value):
    coin_values = (2, 5, 10, 20, 50, 100, 200)
    tally = collections.defaultdict(int)
    tally[(0, float('inf'))] = 1
    new_tally = None
    running_count = 0
    while any(tally):
        new_tally = collections.defaultdict(int)
        for (coin_sum, smallest_coin) in tally:
            running_count += tally[(coin_sum, smallest_coin)]
            for next_coin in coin_values:
                if next_coin <= smallest_coin and coin_sum + next_coin <= max_value:
                    new_tally[(coin_sum + next_coin, next_coin)] += tally[(coin_sum, smallest_coin)]
        tally = new_tally
    return(running_count)

In [15]:
%%time
ways_to_get_sum(200)

CPU times: user 8.86 ms, sys: 1.2 ms, total: 10.1 ms
Wall time: 9.97 ms


73682

In [18]:
# how I solved this in 2018.
# n is the desired sum
# d is the largest denomination allowed to be used

def ways(n,d):
    if n in {0,1} or d == 1:
        return 1
    else:
        wlist = []
        for k in [1,2,5,10,20,50,100,200]:
            if k <= n and k <= d:
                wlist.append(ways(n-k,k))
    return sum(wlist)

In [23]:
%%time
ways(200,200)

CPU times: user 65.6 ms, sys: 2.45 ms, total: 68 ms
Wall time: 68.8 ms


73682

In [24]:
%%time
ways_to_get_sum(1000)

CPU times: user 215 ms, sys: 4.31 ms, total: 220 ms
Wall time: 243 ms


321335886

In [25]:
%%time
ways(1000,200)

CPU times: user 4min 57s, sys: 2.71 s, total: 5min
Wall time: 5min 24s


321335886